In [ ]:
from torch import flatten
from torch.nn import Flatten
import torchvision
import torchvision.transforms as transforms

# transform 정의
transform = transforms.Compose([
    transforms.ToTensor()
])

# 학습 데이터 로드
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True
)
x_train = train_dataset.data.numpy()
x_train = x_train.reshape(x_train.shape[0], -1)

y_train = train_dataset.targets.numpy()
y_train = y_train.reshape(y_train.shape[0], -1)


# 테스트 데이터 로드
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True
)

x_test = test_dataset.data.numpy()
x_test = x_test.reshape(x_test.shape[0], -1)
y_test = test_dataset.targets.numpy()
y_test = y_test.reshape(y_test.shape[0], -1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000, 1)
(10000, 784)
(10000, 1)


In [ ]:
import numpy as np
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

img = x_train[0]
label = y_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
img_show(img)


[5]
(784,)


In [19]:
import numpy as np
import pickle

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    return np.exp(x - c) / np.sum(np.exp(x))

def get_data():
    test_dataset = torchvision.datasets.MNIST(
        root='./data',
        train=False,
        download=True
    )
    x_test = test_dataset.data.numpy()
    x_test = x_test.reshape(x_test.shape[0], -1)
    t_test = test_dataset.targets.numpy()
    t_test = t_test.reshape(t_test.shape[0], -1)
    return x_test, t_test

def init_network():
    with open('./data/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = x.dot(W1) + b1
    z1 = sigmoid(a1)
    a2 = z1.dot(W2) + b2
    z2 = sigmoid(a2)
    a3 = z2.dot(W3) + b3
    y = softmax(a3)
    return y

In [ ]:
# 개별 처리
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9207


/var/folders/b5/twr45y5d2dsfhf3hmqdf8jy80000gn/T/ipykernel_6846/3175279834.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [25]:
# 배치 처리
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:10.5834


/var/folders/b5/twr45y5d2dsfhf3hmqdf8jy80000gn/T/ipykernel_6846/3175279834.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
